# 1. Preparation before running plans... 

## 1.1. Import Numpy and Matplotlib

In [40]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

## 1.2. Create a RunEngine:

In [41]:
from bluesky import RunEngine
RE = RunEngine({})

## 1.3. Prepare Live Visualization¶

In [42]:
from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()

# Send all metadata/data captured to the BestEffortCallback.
RE.subscribe(bec)

# Make plots update live while scans run.
from bluesky.utils import install_kicker
install_kicker()

## 1.4. Prepare Data Storage

In [43]:
from databroker import Broker
db = Broker.named('temp')

# Insert all metadata/data captured into db.
RE.subscribe(db.insert)

1

##### This example makes a temporary database. Do not use it for important data. The data will become difficult to access once Python exits or the variable db is deleted. Running Broker.named('temp') a second time creates a fresh, separate temporary database.

## 1.5. Add a Progress Bar

In [44]:
from bluesky.utils import ProgressBarManager
RE.waiting_hook = ProgressBarManager()

# 2. Connect to a PV from Ophyd¶

## 2.0. For simulation, open a terminal under bluesky-tutorials

~$ python -m caproto.ioc_examples.random_walk --list-pvs

## 2.1. Connect to the PV (random_walk:dt) from Ophyd.

The PV name, random_walk:dt.

A human-friendly name. This name is used to label the readings and will be used in any downstream data analysis or file-writing code. We might choose, for example, time_delta.

In [6]:
from ophyd.signal import EpicsSignal

In [7]:
time_delta = EpicsSignal("random_walk:dt", name="time_delta")

**** The executable "caRepeater" couldn't be located
**** because of errno = "No such file or directory".
**** You may need to modify your PATH environment variable.
**** Unable to start "CA Repeater" process.


In [8]:
from ophyd.signal import EpicsSignalRO  #read-only EpicsSignal

In [9]:
x = EpicsSignalRO("random_walk:x", name="x")

In [19]:
from bluesky import RunEngine
from bluesky.callbacks import LiveTable

RE = RunEngine()
token = RE.subscribe(LiveTable(["time_delta", "x"]))

In [10]:
from bluesky.plans import count, list_scan
RE(count([time_delta]))  # Use as a "detector".



Transient Scan ID: 1     Time: 2022-02-16 12:16:34
Persistent Unique Scan ID: 'dba575a5-de56-4b90-bb9e-42225ceaa7b8'
New stream: 'primary'
+-----------+------------+------------+
|   seq_num |       time | time_delta |
+-----------+------------+------------+
|         1 | 12:16:34.4 |          3 |
+-----------+------------+------------+
generator count ['dba575a5'] (scan num: 1)





('dba575a5-de56-4b90-bb9e-42225ceaa7b8',)

CA client library is unable to contact CA repeater after 50 tries.
Silence this message by starting a CA repeater daemon
or by calling ca_pend_event() and or ca_poll() more often.


In [12]:
RE(list_scan([], time_delta, [0.1, 0.3, 1, 3]))  # Use as "motor".



Transient Scan ID: 2     Time: 2022-02-16 12:17:30
Persistent Unique Scan ID: 'fa42c986-d963-49ad-9b1f-9adc9518f36c'
New stream: 'primary'
+-----------+------------+------------+
|   seq_num |       time | time_delta |
+-----------+------------+------------+
|         1 | 12:17:30.4 |          0 |
|         2 | 12:17:30.4 |          0 |
|         3 | 12:17:30.4 |          1 |
|         4 | 12:17:30.5 |          3 |
+-----------+------------+------------+
generator list_scan ['fa42c986'] (scan num: 2)





('fa42c986-d963-49ad-9b1f-9adc9518f36c',)

In [13]:
from bluesky.plan_stubs import mv
RE(mv(time_delta, 1))

()

In [14]:
RE(count([x], num=5, delay=0.5))  # Read every 0.5 seconds.



Transient Scan ID: 3     Time: 2022-02-16 12:18:52
Persistent Unique Scan ID: '026bac98-5e20-472a-8df3-601b213737ad'
New stream: 'primary'


<IPython.core.display.Javascript object>

+-----------+------------+------------+
|   seq_num |       time |          x |
+-----------+------------+------------+
|         1 | 12:18:52.1 |         -9 |
|         2 | 12:18:52.5 |         -9 |
|         3 | 12:18:53.0 |         -9 |
|         4 | 12:18:53.5 |         -9 |
|         5 | 12:18:54.0 |         -9 |
+-----------+------------+------------+
generator count ['026bac98'] (scan num: 3)





('026bac98-5e20-472a-8df3-601b213737ad',)

but this required us to choose an update frequency (0.5). It’s often better to rely on the control system to tell us when a new value is available. In this example, we accumulate updates for x whenever it changes.

In [17]:
from bluesky.plan_stubs import monitor, unmonitor, open_run, close_run, sleep
def monitor_x_for(duration, md=None):
    yield from open_run(md)  # optional metadata
    yield from monitor(x, name="x_monitor")
    yield from sleep(duration)  # Wait for readings to accumulate.
    yield from unmonitor(x)
    yield from close_run()

In [20]:
RE(monitor_x_for(3), LiveTable(["x"], stream_name="x_monitor"))



Transient Scan ID: 5     Time: 2022-02-16 12:35:36
Persistent Unique Scan ID: '87cc7eb7-3632-465e-bd8d-2154d8e930b3'
New stream: 'x_monitor'


<IPython.core.display.Javascript object>



+-----------+------------+------------+
|   seq_num |       time |          x |
+-----------+------------+------------+
|         1 | 12:35:36.3 |         -6 ||         2 | 12:35:36.3 |         -6 |

|         3 | 12:35:36.5 |         -5 |
|         4 | 12:35:37.5 |         -5 |
|         5 | 12:35:38.5 |         -5 |



+-----------+------------+------------+
generator monitor_x_for ['87cc7eb7'] (scan num: 5)




('87cc7eb7-3632-465e-bd8d-2154d8e930b3',)

### 2.1.1. Read

In [21]:
time_delta.read()

{'time_delta': {'value': 1.0, 'timestamp': 1645031911.771529}}

In [22]:
x.read()

{'x': {'value': -11.225029082119038, 'timestamp': 1645033223.950878}}

### 2.1.2. Describle

In [23]:
time_delta.describe()

{'time_delta': {'source': 'PV:random_walk:dt',
  'dtype': 'number',
  'shape': [],
  'units': '',
  'lower_ctrl_limit': 0.0,
  'upper_ctrl_limit': 0.0,
  'precision': 0}}

### 2.1.3 Set

In [24]:
time_delta.set(10).wait()  # Set it to 10 and wait for it to get there.

In [25]:
time_delta.set(10).wait(timeout=1)  # Set it to 10 and wait up to 1 second.

To move more than one signal in parallel, use the ophyd.status.wait() function.

In [27]:
# from ophyd.status import wait

# # Given signals a and b, set both in motion.
# status1 = a.set(1)
# status2 = b.set(1)
# # Wait for both to complete.
# wait(status1, status2, timeout=1)

### 2.1.4. Subscrible

In [28]:
time_delta.set(1).wait()

In [29]:
from collections import deque

In [30]:
def accumulate(value, old_value, timestamp, **kwargs):
    readings.append({"x": {"value": value, "timestamp": timestamp}})
readings = deque(maxlen=5)
x.subscribe(accumulate)

1

In [31]:
readings

deque([{'x': {'value': -20.696033860354632, 'timestamp': 1645033962.339543}},
       {'x': {'value': -20.16362037083529, 'timestamp': 1645033963.340801}},
       {'x': {'value': -20.275138196450968, 'timestamp': 1645033964.341881}},
       {'x': {'value': -21.158863935300115, 'timestamp': 1645033965.342863}},
       {'x': {'value': -20.9533527307281, 'timestamp': 1645033966.344375}}])

CA.Client.Exception...............................................
    Context: "10.2.234.58:5064"
    Source File: ../cac.cpp line 1237
    Current Time: Wed Feb 16 2022 14:38:13.919794432
..................................................................


## 2.2. Group Signals into Devices

We’ll start two simulated devices that implement a random walk.

$ python -m caproto.ioc_examples.random_walk --prefix="random-walk:horiz:" --list-pvs

$ python -m caproto.ioc_examples.random_walk --prefix="random-walk:vert:" --list-pvs

In [32]:
from ophyd import Component, Device, EpicsSignal, EpicsSignalRO

class RandomWalk(Device):
    x = Component(EpicsSignalRO, 'x')
    dt = Component(EpicsSignal, 'dt')

In [33]:
random_walk_horiz = RandomWalk('random-walk:horiz:', name='random_walk_horiz')

In [34]:
random_walk_horiz.wait_for_connection()
random_walk_horiz

RandomWalk(prefix='random-walk:horiz:', name='random_walk_horiz', read_attrs=['x', 'dt'], configuration_attrs=[])

In [35]:
random_walk_vert = RandomWalk('random-walk:vert:', name='random_walk_vert')

In [36]:
random_walk_vert.wait_for_connection()
random_walk_vert

RandomWalk(prefix='random-walk:vert:', name='random_walk_vert', read_attrs=['x', 'dt'], configuration_attrs=[])

In [52]:
from bluesky import RunEngine
from bluesky.callbacks import LiveTable

RE = RunEngine()
token = RE.subscribe(LiveTable(["random_walk_horiz_x", "random_walk_horiz_dt"]))

In [53]:
from bluesky.plans import count

RE(count([random_walk_horiz.x], num=3, delay=1))



+-----------+------------+---------------------+
|   seq_num |       time | random_walk_horiz_x |
+-----------+------------+---------------------+
|         1 | 14:47:57.9 |                   1 |
|         2 | 14:47:58.9 |                   1 |
|         3 | 14:47:59.9 |                   1 |
+-----------+------------+---------------------+
generator count ['4465819b'] (scan num: 1)




('4465819b-9eb7-4545-8ad7-1af3e79973e7',)

In [54]:
RE(count([random_walk_horiz], num=3, delay=1))



+-----------+------------+---------------------+----------------------+
|   seq_num |       time | random_walk_horiz_x | random_walk_horiz_dt |
+-----------+------------+---------------------+----------------------+
|         1 | 14:48:01.9 |                   1 |                    3 |
|         2 | 14:48:02.9 |                   2 |                    3 |
|         3 | 14:48:03.9 |                   2 |                    3 |
+-----------+------------+---------------------+----------------------+
generator count ['514e997e'] (scan num: 2)




('514e997e-843c-480b-9681-bd6759b7623f',)

In [55]:
random_walk_horiz.read()

OrderedDict([('random_walk_horiz_x',
              {'value': -4.889727269899685, 'timestamp': 1645040975.605327}),
             ('random_walk_horiz_dt',
              {'value': 3.0, 'timestamp': 1645040333.373043})])

In [72]:
random_walk_horiz.describe()

OrderedDict([('random_walk_horiz_x',
              {'source': 'PV:random-walk:horiz:x',
               'dtype': 'number',
               'shape': [],
               'units': '',
               'lower_ctrl_limit': 0.0,
               'upper_ctrl_limit': 0.0,
               'precision': 0}),
             ('random_walk_horiz_dt',
              {'source': 'PV:random-walk:horiz:dt',
               'dtype': 'number',
               'shape': [],
               'units': '',
               'lower_ctrl_limit': 0.0,
               'upper_ctrl_limit': 0.0,
               'precision': 0})])

## 2.3. Ready-to-Use Devices : EpicsMotor

In [78]:
from ophyd import EpicsMotor

hdt = EpicsMotor('random-walk:horiz:dt', name='horiz_dt')

In [79]:
hdt.prefix

'random-walk:horiz:dt'

In [80]:
hdt.wait_for_connection()

TimeoutError: Failed to connect to all signals: horiz_dt.user_readback (random-walk:horiz:dt.RBV), horiz_dt.user_offset (random-walk:horiz:dt.OFF), horiz_dt.user_offset_dir (random-walk:horiz:dt.DIR), horiz_dt.offset_freeze_switch (random-walk:horiz:dt.FOFF), horiz_dt.set_use_switch (random-walk:horiz:dt.SET), horiz_dt.velocity (random-walk:horiz:dt.VELO), horiz_dt.acceleration (random-walk:horiz:dt.ACCL), horiz_dt.motor_egu (random-walk:horiz:dt.EGU), horiz_dt.motor_is_moving (random-walk:horiz:dt.MOVN), horiz_dt.motor_done_move (random-walk:horiz:dt.DMOV), horiz_dt.high_limit_switch (random-walk:horiz:dt.HLS), horiz_dt.low_limit_switch (random-walk:horiz:dt.LLS), horiz_dt.high_limit_travel (random-walk:horiz:dt.HLM), horiz_dt.low_limit_travel (random-walk:horiz:dt.LLM), horiz_dt.direction_of_travel (random-walk:horiz:dt.TDIR), horiz_dt.motor_stop (random-walk:horiz:dt.STOP), horiz_dt.home_forward (random-walk:horiz:dt.HOMF), horiz_dt.home_reverse (random-walk:horiz:dt.HOMR); Pending 